In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

import geopandas as gpd
import pandas as pd
import folium 
import branca
import datetime as dt
from siuba import *
from bus_service_utils import better_bus_utils

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

# 100 Recs - D11

## 1 - Corridor Improvements

In [ ]:
corridor_gdf_d11 = ((better_bus_utils.get_sorted_transit_routes(recommendation_category="corridor"))
                   >>filter(_.caltrans_district=="11 - San Diego")
                   >>head(10) #get top 10
                  )

In [ ]:
corridor_gdf_d11

In [ ]:
corridor_gdf_d11 >> distinct(_.calitp_itp_id)

In [ ]:
corridor_d11 = (corridor_gdf_d11
               >> select(-_.service_date)
              )
corridor_d11.explore("route_id", legend=True, tiles="CartoDB positron")

In [ ]:
# for each transit operator, generate rtfiltermapper
sdmts = rt_filter_map_plot.from_gcs(itp_id=278, analysis_date=dt.date(2022,10,12))

In [7]:
# get small df of route_id and route_short_name from rt_trips
d11_routenames = (sdmts.rt_trips
                 >> distinct(_.route_id, _.route_short_name)
                 >> inner_join(_, corridor_gdf_d11, on = 'route_id')
                )
d11_routenames

,route_id,route_short_name,calitp_itp_id,category,geometry,service_date,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district
0,204,204,278,intersects_shn,"LINESTRING (-117.21380 32.86834, -117.21384 32...",2022-05-04,17.0,33.0,None,NaN,short,3.0,0.0,0.0,8.309091,2.384848,11 - San Diego
1,27,27,278,intersects_shn,"LINESTRING (-117.13467 32.83063, -117.13474 32...",2022-05-04,38.0,62.0,None,NaN,short,19.2,0.0,0.0,29.245161,2.804516,11 - San Diego
2,12,12,278,intersects_shn,"LINESTRING (-117.02645 32.69715, -117.02933 32...",2022-05-04,72.0,128.0,None,NaN,short,13.2,0.0,0.0,15.825000,1.550469,11 - San Diego
3,237,237,278,intersects_shn,"LINESTRING (-117.11987 32.91223, -117.12029 32...",2022-05-04,52.0,57.0,None,NaN,short,15.6,0.0,0.0,18.305263,1.953684,11 - San Diego
4,14,14,278,intersects_shn,"LINESTRING (-117.03313 32.78148, -117.03328 32...",2022-05-04,18.0,26.0,None,NaN,short,6.0,0.0,0.0,26.192308,2.454615,11 - San Diego
5,2,2,278,intersects_shn,"LINESTRING (-117.16580 32.71882, -117.16473 32...",2022-05-04,72.0,136.0,None,NaN,short,12.0,0.0,0.0,12.917647,1.654853,11 - San Diego
6,10,10,278,intersects_shn,"LINESTRING (-117.06632 32.75228, -117.06447 32...",2022-05-04,73.0,129.0,None,NaN,short,18.0,0.0,0.0,20.288372,1.736434,11 - San Diego
7,28,28,278,intersects_shn,"LINESTRING (-117.19986 32.75480, -117.19968 32...",2022-05-04,47.0,80.0,None,NaN,short,9.0,0.0,0.0,10.327500,2.229000,11 - San Diego
8,201,201,278,intersects_shn,"LINESTRING (-117.21340 32.86857, -117.21328 32...",2022-05-04,78.0,133.0,None,NaN,short,17.4,0.0,0.0,16.430075,2.827293,11 - San Diego


In [45]:
# inner join w gdf of slow routes
corridor_d11_names = (corridor_d11
                     >> inner_join(_,d11_routenames)
                    )

corridor_d11_names >> head(3)

,calitp_itp_id,route_id,category,geometry,trips_peak,trips_all_day,route_type,mean_speed_mph,route_group,bus_difference_spread,num_competitive,pct_trips_competitive,bus_difference_mean,bus_multiplier_mean,caltrans_district,route_short_name,route_long_name
0,278,10,intersects_shn,"LINESTRING (-117.06632 32.75228, -117.06447 32...",73.0,129.0,None,NaN,short,18.0,0.0,0.0,20.288372,1.736434,11 - San Diego,10,Old Town - University/College
1,278,12,intersects_shn,"LINESTRING (-117.02645 32.69715, -117.02933 32...",72.0,128.0,None,NaN,short,13.2,0.0,0.0,15.825000,1.550469,11 - San Diego,12,City College - Skyline Hills
2,278,14,intersects_shn,"LINESTRING (-117.03313 32.78148, -117.03328 32...",18.0,26.0,None,NaN,short,6.0,0.0,0.0,26.192308,2.454615,11 - San Diego,14,Grantville Trolley - Baltimore & Lake Murray


In [9]:
# pass the route names as a filter parameter
routename_list_c = corridor_d11_names["route_short_name"].tolist()
routename_list_c

['10', '12', '14', '2', '201', '204', '237', '27', '28']

In [10]:
sdmts.set_filter(start_time='06:00', end_time='09:00', route_names = routename_list_c)

In [11]:
am = sdmts.segment_speed_map()

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:394: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:394: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [12]:
am

In [13]:
sdmts.set_filter(start_time='15:00', end_time='19:00', route_names = routename_list_c)

In [14]:
pm = sdmts.segment_speed_map()

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:394: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:394: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [15]:
pm

## Draft Corridor Recs, SDMTS, D11

* Entire Route 28 (nn mi, nn speed-based delay, nn schedule-based delay)
* Route 10, Goldfinch to 54 ()
    * paralleled by faster Rapid 215 E of Park
* Route 27, Mission to Moraga
* Route 12, Broadway to Euclid

## Route 28

In [16]:
shape_id = '28_2_48'
stop_range = [2, 18]

In [17]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [18]:
sdmts.reset_filter()

In [19]:
sdmts.corridor_metrics()

{'schedule_metric_minutes': 129.65833333333333,
 'speed_metric_minutes': 1537.8479416031244}

In [20]:
sdmts.quick_map_corridor()

In [21]:
corr_28 = sdmts.corridor.copy()

In [22]:
corridors = [corr_28]

In [23]:
# sdmts.corridor.explore(m = pm)

## Route 10

In [24]:
shape_id = '10_2_68'
stop_range = [6, 21]

In [25]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [26]:
sdmts.reset_filter()

In [27]:
sdmts.corridor_metrics()

{'schedule_metric_minutes': 461.35, 'speed_metric_minutes': 4748.958087052544}

In [28]:
sdmts.quick_map_corridor()

In [29]:
corr_10 = sdmts.corridor.copy()
corridors += [corr_10]

## Route 27

In [30]:
shape_id = '27_3_87'
stop_range = [16, 31]

In [31]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [33]:
sdmts.reset_filter()

In [34]:
sdmts.corridor_metrics()

{'schedule_metric_minutes': -61.65833333333333,
 'speed_metric_minutes': -946.0625551819488}

In [35]:
sdmts.quick_map_corridor()

In [36]:
corr_27 = sdmts.corridor.copy()
corridors += [corr_27]

## Route 12

In [37]:
shape_id = '12_2_13'
stop_range = [3, 31]

In [38]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [39]:
sdmts.set_filter(route_names=['12'])

In [40]:
sdmts.corridor_metrics()

{'schedule_metric_minutes': 81.44166666666666,
 'speed_metric_minutes': 1212.8478495641455}

In [41]:
sdmts.quick_map_corridor()

In [42]:
corr_12 = sdmts.corridor.copy()
corridors += [corr_12]

# Corridor Map

In [43]:
pd.concat(corridors).explore(m = pm)

## 2 - Hotspot Improvements

In [ ]:
hotspot_gdf_d11 = ((better_bus_utils.get_sorted_transit_routes(recommendation_category="hotspot"))
                   >>filter(_.caltrans_district=="11 - San Diego")
                   >>head(10) #get top 10
                  )

In [ ]:
# hotspot_gdf_d11

In [ ]:
hotspot_gdf_d11 = (hotspot_gdf_d11
               >> select(-_.service_date)
              )

hotspot_gdf_d11.explore("route_id", legend=True, tiles="CartoDB positron")

In [ ]:
# inner join w gdf of slow routes
hs_d11_names = (hotspot_gdf_d11
                     >> inner_join(_,d11_routenames)
                    )

hs_d11_names

In [ ]:
# pass the route names as a filter parameter
routename_list_hs = hs_d11_names["route_short_name"].tolist()
routename_list_hs

In [ ]:
sdmts.set_filter(start_time='06:00', end_time='09:00', route_names = routename_list_hs)

In [ ]:
am = sdmts.segment_speed_map()

In [ ]:
am

In [ ]:
sdmts.set_filter(start_time='15:00', end_time='19:00', route_names = routename_list_hs)

In [ ]:
pm = sdmts.segment_speed_map()

In [ ]:
pm

# Draft Hotspot Recs, SDMTS, D11

## Route 27 (Genesee)

In [ ]:
shape_id = '27_2_86'
stop_range = [19, 23]

In [ ]:
sdmts.reset_filter()

In [ ]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
sdmts.corridor_metrics()

In [ ]:
sdmts.quick_map_corridor()